In [10]:
import pandas as pd
data = pd.read_csv('train.csv', nrows = 200000)
test = pd.read_csv('test.csv', nrows = 200000)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
#  счетчики со сглаживанием
from collections import defaultdict
def counts(cat_col, targ_col, class_val, c):
    d = defaultdict(lambda: [0,0])
    for cat_val, target_val in zip(cat_col, targ_col):
        d[cat_val][0] += int(target_val == class_val)
        d[cat_val][1] += 1
    
    for k in d.keys():
        d[k] = (d[k][0] + c) / (d[k][1] + c)
    
    return dict(d)

In [12]:
# Для многлассовой задачи
def cat_encode_all(cat_col, target_col, const):
    t_uniq = sorted(np.unique(target_col))
    mappers = dict()
    for val in t_uniq:
        print(val)
        mappers[val] = counts(cat_col, target_col, val, const)
    return mappers

In [13]:
X_train = data.drop('HasDetections', axis = 1)
y_train = data['HasDetections']
all_cat = ['EngineVersion']
X_test = test.copy()

In [14]:
from sklearn.model_selection import StratifiedKFold
import numpy as np
skf = StratifiedKFold(n_splits=4)
skf.get_n_splits(X_train, y_train)

print(skf)  

for cat_col in all_cat:
    for train_index, test_index in skf.split(X_train, y_train):
        fold_X_train, fold_X_test = X_train.loc[train_index], X_train.loc[test_index]
        fold_y_train, fold_y_test = y_train.loc[train_index], y_train.loc[test_index]
        c = cat_encode_all(fold_X_train[cat_col], fold_y_train, 0)
        for i in c.keys():
            if i != 0:
                X_train.loc[test_index, cat_col + '_counts_' + str(i)] = fold_X_test[cat_col].map(c[i]).fillna((y_train == i).mean())
    
    train_c = cat_encode_all(X_train[cat_col].values, y_train.values, 0)
    for i in train_c.keys():
        if i != 0:
            X_test[cat_col + '_counts_' + str(i)] = X_test[cat_col].map(train_c[i]).fillna((y_train == i).mean())
    
    print(cat_col, '\n')

X_train.drop(all_cat, axis=1, inplace=True)
X_test.drop(all_cat, axis=1, inplace=True)

StratifiedKFold(n_splits=4, random_state=None, shuffle=False)
0
1
0
1
0
1
0
1
0
1
EngineVersion 



In [15]:
X_train

,MachineIdentifier,ProductName,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,EngineVersion_counts_1
0,0000028988387b115f69f31a3bf04f09,win8defender,4.18.1807.18075,1.273.1735.0,0,7.0,0,NaN,53447.0,1.0,...,36144.0,0,NaN,0.0,0,0,0.0,0.0,10.0,0.551575
1,000007535c3f730efa9ea0b7ef1bd645,win8defender,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,53447.0,1.0,...,57858.0,0,NaN,0.0,0,0,0.0,0.0,8.0,0.448538
2,000007905a28d863f6d0d597892cd692,win8defender,4.18.1807.18075,1.273.1341.0,0,7.0,0,NaN,53447.0,1.0,...,52682.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0.551575
3,00000b11598a75ea8ba1beea8459149f,win8defender,4.18.1807.18075,1.273.1527.0,0,7.0,0,NaN,53447.0,1.0,...,20050.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0.551575
4,000014a5f00daa18e76b81417eeb99fc,win8defender,4.18.1807.18075,1.273.1379.0,0,7.0,0,NaN,53447.0,1.0,...,19844.0,0,0.0,0.0,0,0,0.0,0.0,1.0,0.551575
5,000016191b897145d069102325cab760,win8defender,4.18.1807.18075,1.273.1094.0,0,7.0,0,NaN,53447.0,1.0,...,51039.0,0,0.0,0.0,0,0,0.0,0.0,15.0,0.551575
6,0000161e8abf8d8b89c5ab8787fd712b,win8defender,4.18.1807.18075,1.273.845.0,0,7.0,0,NaN,43927.0,2.0,...,63175.0,1,NaN,0.0,0,0,0.0,0.0,10.0,0.551575
7,000019515bc8f95851aff6de873405e8,win8defender,4.18.1807.18075,1.273.1393.0,0,7.0,0,NaN,53447.0,1.0,...,63122.0,0,0.0,0.0,0,0,0.0,0.0,15.0,0.551575
8,00001a027a0ab970c408182df8484fce,win8defender,4.18.1807.18075,1.275.988.0,0,7.0,0,NaN,53447.0,1.0,...,15510.0,0,0.0,0.0,0,0,0.0,0.0,15.0,0.494203
9,00001a18d69bb60bda9779408dcf02ac,win8defender,4.18.1807.18075,1.273.973.0,0,7.0,0,NaN,46413.0,2.0,...,63555.0,1,0.0,0.0,0,0,0.0,1.0,8.0,0.551575


In [16]:
X_test

,MachineIdentifier,ProductName,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,EngineVersion_counts_1
0,0000010489e3af074adeac69c53e555e,win8defender,4.18.1810.5,1.281.501.0,0,7.0,0,NaN,53447.0,1.0,...,8554.0,1,NaN,0.0,0,0,0.0,0.0,7.0,0.500300
1,00000176ac758d54827acd545b6315a5,win8defender,4.18.1809.2,1.279.301.0,0,7.0,0,NaN,53447.0,1.0,...,33105.0,1,NaN,0.0,0,0,0.0,1.0,12.0,0.500300
2,0000019dcefc128c2d4387c1273dae1d,win8defender,4.18.1809.2,1.277.230.0,0,7.0,0,NaN,49480.0,2.0,...,63396.0,1,NaN,0.0,0,0,0.0,1.0,11.0,0.487001
3,0000055553dc51b1295785415f1a224d,win8defender,4.18.1810.5,1.281.664.0,0,7.0,0,NaN,42160.0,2.0,...,26320.0,1,0.0,0.0,0,0,0.0,0.0,10.0,0.500300
4,00000574cefffeca83ec8adf9285b2bf,win8defender,4.18.1809.2,1.279.236.0,0,7.0,0,NaN,53447.0,1.0,...,63269.0,1,NaN,0.0,0,0,0.0,1.0,3.0,0.500300
5,000007ffedd31948f08e6c16da31f6d1,win8defender,4.18.1809.2,1.277.724.0,0,7.0,0,NaN,53447.0,1.0,...,36579.0,0,NaN,0.0,0,0,0.0,0.0,10.0,0.487001
6,000008f31610018d898e5f315cdf1bd1,win8defender,4.18.1810.5,1.279.1373.0,0,7.0,0,NaN,7945.0,2.0,...,52312.0,1,NaN,0.0,0,0,0.0,1.0,10.0,0.500300
7,00000a3c447250626dbcc628c9cbc460,win8defender,4.18.1806.18062,1.277.1185.0,0,7.0,0,NaN,15521.0,2.0,...,1692.0,0,NaN,0.0,0,0,0.0,0.0,9.0,0.487001
8,00000b6bf217ec9aef0f68d5c6705897,win8defender,4.18.1810.5,1.281.675.0,0,7.0,0,NaN,53447.0,1.0,...,NaN,0,NaN,0.0,0,0,NaN,NaN,NaN,0.500300
9,00000b8d3776b13e93ad83676a28e4aa,win8defender,4.14.17613.18039,1.265.676.0,0,7.0,0,NaN,53447.0,1.0,...,NaN,0,0.0,0.0,0,0,0.0,0.0,15.0,0.342493


# Для каждой кат фичи получаем кол-во колонок = n-1, где n - кол-во уникальных значений таргета